![brainome logo](./images/brainome_logo.png)
# 104 Using Brainome's Predictor CLI
The predictor generated by Brainome is capable of being used by the command line interface (CLI).


1. Predictor --help
2. Validate test csv dataset
3. Classify unlabeled csv dataset
4. Feature engineering predictions

## Prerequisites
This notebook assumes brainome is installed as per notebook [brainome_101_Quick_Start](brainome_101_Quick_Start.ipynb)

The data sets are:
* [titanic_train.csv](https://download.brainome.ai/data/public/titanic_train.csv) for training data
* [titanic_validate.csv](https://download.brainome.ai/data/public/titanic_validate.csv) for validation
* [titanic_predict.csv](https://download.brainome.ai/data/public/titanic_predict.csv) for predictions

In [ ]:
!brainome -version

import urllib.request as request
response1 = request.urlretrieve('https://download.brainome.ai/data/public/titanic_train.csv', 'titanic_train.csv')
response2 = request.urlretrieve('https://download.brainome.ai/data/public/titanic_validate.csv', 'titanic_validate.csv')
response3 = request.urlretrieve('https://download.brainome.ai/data/public/titanic_predict.csv', 'titanic_predict.csv')
%ls -lh titanic_train.csv titanic_validate.csv titanic_predict.csv


## Generate a predictor

In [ ]:
!brainome titanic_train.csv -rank -y -o predictor_104.py -modelonly -q

The predictor file is [predictor_104.py](predictor_104.py)

## 1. Predictor help
Brainome predictors are really short and sweet. They just validate and classify data.

While the predictor source code is portable, it does require numpy to run and optionally scipy to generate the confusion matrices.

In [ ]:
!python3 predictor_104.py --help

## 2. Validate test dataset
The validate function takes a csv data set identical to the training data set and, with the `-validate` parameter, compares outcomes.

In [ ]:
!python3 predictor_104.py -validate titanic_validate.csv

## 3. Classify unlabeled dataset
The predictor can classify a similar to training/validation data set sans target column.

It will generate a complete data set with the "Prediction" column appended.

In [ ]:
!python3 predictor_104.py titanic_predict.csv > classifications_104.csv
print('Viewing classification predictions.')
import pandas as pd
pd.read_csv('classifications_104.csv')


## 4. Feature engineering predictions
While feature engineering, it is desired to only view the features that contributed to the prediction. 

With the `-trim` parameter, the output will only show the features deemed important by the model.

In [ ]:
!python3 predictor_104.py titanic_predict.csv -trim > trimmed_classifications_104.csv
print('Viewing important features classification predictions.')
# preview uses pandas to read and display csv data
import pandas as pd
pd.read_csv('trimmed_classifications_104.csv')

## Advanced Predictor Usage
See notebook [300 Put your model to work](./brainome_300_Integrating_Predictors.ipynb) for integrating the predictor within your python program.

## Next Steps
- Check out [106 Describe Your CSV](./brainome_106_Describe_Your_CSV.ipynb)
- Check out [Using Measurement to Create Better Models](./brainome_200_Using_Measurement.ipynb)